In [1]:
import os
import numpy as np

from sunpy.net import Fido, attrs as a
import astropy.units as u

import sunpy.map
from astropy.io import fits

from aiapy.calibrate import register, update_pointing, correct_degradation
from aiapy.calibrate.util import get_pointing_table, get_correction_table
import aiapy.psf

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import cv2
from PIL import Image

import shutil

import time
import datetime
from dateutil.relativedelta import relativedelta

from concurrent.futures import ThreadPoolExecutor



### Downloading the Data

#### Set time range

In [2]:
start_date = '2013/01/01 00:00:00'
end_date = '2013/01/02 00:00:00'

# Set Time Range
time_range = a.Time(start_date, end_date)

#### Download Directory

In [3]:
# Specify the directory where you want to download the files
download_directory = 'C:/Sun Data/'
os.makedirs(download_directory, exist_ok=True)

#### Search for fits on VSO

In [4]:


all_results = []

start_date_dt = datetime.datetime.strptime(start_date, '%Y/%m/%d %H:%M:%S')
end_date_dt = datetime.datetime.strptime(end_date, '%Y/%m/%d %H:%M:%S')

current_date = start_date_dt

while current_date < end_date_dt:
    next_date = current_date + relativedelta(months=1)
    if next_date > end_date_dt:
        next_date = end_date_dt

    time_range = a.Time(current_date.strftime('%Y/%m/%d %H:%M:%S'), next_date.strftime('%Y/%m/%d %H:%M:%S'))
    print("Searching for: ", time_range)
    results = Fido.search(
        time_range,
        a.Instrument.aia,
        a.Wavelength(193 * u.angstrom),
        a.Sample(60 * u.minute)
    )
    print("Found", results.file_num, "results")
    all_results.extend(results)
    current_date = next_date


print(all_results)


Searching for:  <sunpy.net.attrs.Time(2013-01-01 00:00:00.000, 2013-02-01 00:00:00.000)>


KeyboardInterrupt: 

#### Download files with error checking

In [5]:
for results in all_results:
    downloaded_files = []
    while results.file_num != len(downloaded_files):
        try:
            downloaded_files = Fido.fetch(
                results, path=download_directory)
        except Exception as e:
            print(
                f"Failed to download files. Retrying...")
            continue

# Output the list of downloaded files
print(f"Downloaded {len(downloaded_files)} files.")

Files Downloaded:   0%|          | 0/24 [00:00<?, ?file/s]

aia.lev1.193A_2024_01_01T01_00_04.84Z.image_lev1.fits:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

aia.lev1.193A_2024_01_01T02_00_04.85Z.image_lev1.fits:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

aia.lev1.193A_2024_01_01T03_00_04.85Z.image_lev1.fits:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

aia.lev1.193A_2024_01_01T00_00_07.05Z.image_lev1.fits:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

...

aia.lev1.193A_2024_01_01T20_00_04.84Z.image_lev1.fits:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

aia.lev1.193A_2024_01_01T21_00_02.01Z.image_lev1.fits:   0%|          | 0.00/6.76M [00:00<?, ?B/s]

aia.lev1.193A_2024_01_01T22_00_04.84Z.image_lev1.fits:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

aia.lev1.193A_2024_01_01T23_00_04.84Z.image_lev1.fits:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

KeyboardInterrupt: 

### Quality checking and Re-downloading

#### First we check quality of all files, if the quality is less than 0 we delete the file and save the date and time in an array.
#### Then we redownload files 5 minutes after those files 
#### Then we re-check the quality of those newly downloaded files
#### Then we delete all the bad quality files

In [7]:

def check_file_quality(file_list, download_directory):
    bad_quality_files = []
    # open all files and check the quality
    for file in file_list:
        try:
            with fits.open(file) as hdul:
                header = hdul[1].header
                # Check if the quality is bad add to the list
                if header['QUALITY'] != 0:
                    bad_quality_files.append(file)
                    print(f"Bad quality file: {file}")
                hdul.close()
        except Exception as e:
            print(f"Failed to open {file}: {e}")
            continue
    # delete all bad quality files
    for bad_file in bad_quality_files:
        os.remove(bad_file)
        
    # Save the list of bad quality files to a text file
    bad_quality_files_file = os.path.join(download_directory, 'bad_quality_files.txt')
    with open(bad_quality_files_file, 'w') as f:
        for file in bad_quality_files:
            f.write(f"{file}\n")
            
    # return the list of bad quality files
    return bad_quality_files

def download_list_of_files(file_list, download_directory):
    for bad_file in file_list:
        # To remove path
        bad_file_list = bad_file.split('\\')
        # Get just the file name
        bad_file = bad_file_list[-1]
        # Get the date and time of the bad quality file
        year = bad_file[14:18]
        month = bad_file[19:21]
        day = bad_file[22:24]
        hour = bad_file[25:27]
        
        # Create a new date with the same date but 5 minutes later
        new_date = f'{year}/{month}/{day} {hour}:05:00'
        new_date_10 = f'{year}/{month}/{day} {hour}:10:00'

        # Download the new file for the bad quality file
        new_time_range = a.Time(new_date, new_date_10)
        new_results = Fido.search(
            new_time_range,
            a.Instrument.aia,
            a.Wavelength(193 * u.angstrom),
            a.Sample(60 * u.minute)
        )

        # print(new_results)

        # Download the new data
        new_downloaded_files = Fido.fetch(new_results, path=download_directory)

        # Save the list of newly downloaded files to a text file
        new_downloaded_files_file = os.path.join(download_directory, 'new_downloaded_files.txt')
        with open(new_downloaded_files_file, 'a') as f:
            for file in new_downloaded_files:
                f.write(f"{file}\n")
        


# check and delete bad quality files
bad_quality_files = check_file_quality(downloaded_files, download_directory)
# download new files for the bad quality files
download_list_of_files(bad_quality_files, download_directory)

# Open the new_downloaded_files.txt and read the contents
new_downloaded_files_file = os.path.join(download_directory, 'new_downloaded_files.txt')
with open(new_downloaded_files_file, 'r') as f:
    new_downloaded_files = f.readlines()

# Remove the newline character from the file names
new_downloaded_files = [file.strip() for file in new_downloaded_files]

# Print the list of newly downloaded files
print("Newly downloaded files:")
for file in new_downloaded_files:
    print(file)

# check and delete bad quality files
bad_quality_files = check_file_quality(new_downloaded_files, download_directory)

print("New files with bad quality:")
for file in bad_quality_files:
    print(file)

NameError: name 'downloaded_files' is not defined

### Preprocessing 

#### Pointing correction

#### Pointing correction can be done together with image registration 

In [7]:

# # List all files in the download directory
# aia_files = [f for f in os.listdir(download_directory) if f.endswith('.fits')]

# # update pointing table for all fits files in the download directory
# for aia_file in tqdm(aia_files, desc="Updating pointing"):
#     aia_file_path = os.path.join(download_directory, aia_file)
#     # Open the FITS file
#     try:
#         hdul = fits.open(aia_file_path, mode="update")
#         header = hdul[1].header
#         data = hdul[1].data
        
#     except Exception as e:
#         print(f"FILE CORRUPTED: {aia_file_path}, Error: {e}")
#         continue
#     # remove these two header keywords because they are nan and are causing errors with sunpy map
#     header.remove('OSCNMEAN', ignore_missing=True)
#     header.remove('OSCNRMS', ignore_missing=True)
#     # convert to sunpy map
#     aia_map = sunpy.map.Map((data, header))
#     # download pointing table
#     pointing_table = get_pointing_table("JSOC", time_range=(aia_map.date - 12 * u.h, aia_map.date + 12 * u.h))
#     # update pointing
#     aia_map_updated_pointing = update_pointing(aia_map, pointing_table=pointing_table)
#     hdul[1].header.update(aia_map_updated_pointing.meta)
#     hdul.close()
    
    



#### Define the list of all fits files in the directory

In [4]:
# List all files in the download directory
aia_files = [f for f in os.listdir(download_directory) if f.endswith('.fits')]

#### PSF Deconvolution

In [ ]:

def deconvolve_file(aia_file):
    aia_file_path = os.path.join(download_directory, aia_file)
    try:
        hdul = fits.open(aia_file_path, mode="update")
        header = hdul[1].header
        data = hdul[1].data
    except Exception as e:
        return f"FILE CORRUPTED: {aia_file_path}, Error: {e}"

    header.remove('OSCNMEAN', ignore_missing=True)
    header.remove('OSCNRMS', ignore_missing=True)

    aia_map = sunpy.map.Map((data, header))
    psf = aiapy.psf.psf(aia_map.wavelength)
    aia_map_deconvolved = aiapy.psf.deconvolve(aia_map, psf=psf)

    hdul[1].data = aia_map_deconvolved.data
    hdul.close()
    return None

with ThreadPoolExecutor(max_workers=3) as executor:
    list(tqdm(executor.map(deconvolve_file, aia_files), total=len(aia_files), desc="PSF Deconvolution"))


PSF Deconvolution:   0%|          | 0/24 [00:00<?, ?it/s]

### Pointing Correction and Image Registration 

In [5]:
def process_file(aia_file):
    aia_file_path = os.path.join(download_directory, aia_file)
    try:
        hdul = fits.open(aia_file_path, mode="update")
        header = hdul[1].header
        data = hdul[1].data
    except Exception as e:
        return f"FILE CORRUPTED: {aia_file_path}, Error: {e}"
    header.remove('OSCNMEAN', ignore_missing=True)
    header.remove('OSCNRMS', ignore_missing=True)
    aia_map = sunpy.map.Map((data, header))
    pointing_table = get_pointing_table("JSOC", time_range=(aia_map.date - 12 * u.h, aia_map.date + 12 * u.h))
    aia_map_updated_pointing = update_pointing(aia_map, pointing_table=pointing_table)
    aia_map_registered = register(aia_map_updated_pointing)

    hdul[1].data = aia_map_registered.data
    hdul.close()
    return None

with ThreadPoolExecutor(max_workers=3) as executor:
    list(tqdm(executor.map(process_file, aia_files), total=len(aia_files), desc="Pointing Correction and Image Registration"))


Pointing Correction and Image Registration:   0%|          | 0/24 [00:00<?, ?it/s]

### Degradation Correction

In [6]:
correction_table = get_correction_table("JSOC")
print("Got correction table")

def correct_degradation_file(aia_file):
    aia_file_path = os.path.join(download_directory, aia_file)
    try:
        hdul = fits.open(aia_file_path, mode="update")
        header = hdul[1].header
        data = hdul[1].data
    except Exception as e:
        return f"FILE CORRUPTED: {aia_file_path}, Error: {e}"

    aia_map = sunpy.map.Map((data, header))
    aia_map_corrected = correct_degradation(aia_map, correction_table=correction_table)
    hdul[1].data = aia_map_corrected.data
    hdul.close()
    return None

with ThreadPoolExecutor(max_workers=3) as executor:
    list(tqdm(executor.map(correct_degradation_file, aia_files), total=len(aia_files), desc="Degradation Correction"))


Got correction table


Degradation Correction:   0%|          | 0/24 [00:00<?, ?it/s]

### Exposure Normalization

In [7]:
def normalize_by_exposure(aia_file):
    aia_file_path = os.path.join(download_directory, aia_file)
    try:
        hdul = fits.open(aia_file_path, mode="update")
        header = hdul[1].header
        data = hdul[1].data
    except Exception as e:
        return f"FILE CORRUPTED: {aia_file_path}, Error: {e}"
    
    aia_map = sunpy.map.Map((data, header))
    aia_map_normalized = aia_map / aia_map.exposure_time
    hdul[1].data = aia_map_normalized.data
    hdul.close()
    return None

with ThreadPoolExecutor(max_workers=3) as executor:
    list(tqdm(executor.map(normalize_by_exposure, aia_files), total=len(aia_files), desc="Normalizing by Exposure Time"))


Normalizing by Exposure Time:   0%|          | 0/24 [00:00<?, ?it/s]

c:\Users\mirzi\miniconda3\envs\sundata\lib\site-packages\astropy\units\quantity.py:671: RuntimeWarning: divide by zero encountered in divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
c:\Users\mirzi\miniconda3\envs\sundata\lib\site-packages\astropy\units\quantity.py:671: RuntimeWarning: invalid value encountered in multiply
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


#### Clipping values between 100 and 5000

In [8]:
def clip_values(aia_file):
    aia_file_path = os.path.join(download_directory, aia_file)
    try:
        hdul = fits.open(aia_file_path, mode="update")
        header = hdul[1].header
        data = hdul[1].data
    except Exception as e:
        return f"FILE CORRUPTED: {aia_file_path}, Error: {e}"
    
    # Clip the values between 100 and 5000
    data_clipped = data.clip(100, 5000)
    
    # Save the clipped image
    hdul[1].data = data_clipped
    hdul.close()
    return None

with ThreadPoolExecutor(max_workers=3) as executor:
    list(tqdm(executor.map(clip_values, aia_files), total=len(aia_files), desc="Clipping Values"))

Clipping Values:   0%|          | 0/24 [00:00<?, ?it/s]

#### Rescaling Images by Log10 (Cant do this because it causes issues while copying)

In [ ]:
# for aia_file in tqdm(aia_files, desc="Rescaling to log10"):
#     aia_file_path = os.path.join(download_directory, aia_file)
#     # Open the FITS file
#     try:
#         hdul = fits.open(aia_file_path, mode="update")
#         header = hdul[1].header
#         data = hdul[1].data
#     except Exception as e:
#         print(f"FILE CORRUPTED: {aia_file_path}, Error: {e}")
#         continue
    
#     # Rescale the data to log10 scale
#     data_log10 = np.log10(data + 1)  # Adding 1 to avoid log(0)
    
#     # Save the rescaled image
#     hdul[1].data = data_log10
    
#     hdul.close()

#### Resizing images to 224 x 224 using pillow/opencv with bilinear interpolation

In [9]:

for aia_file in tqdm(aia_files, desc="Downscaling to 224x224"):
    aia_file_path = os.path.join(download_directory, aia_file)
    # Open the FITS file
    # Specify the new directory where you want to copy the files
    new_directory = os.path.join(download_directory, 'non_anti_aliasing/')
    os.makedirs(new_directory, exist_ok=True)

    # Copy the file to the new directory
    new_file_path = os.path.join(new_directory, aia_file)
    shutil.copy2(aia_file_path, new_file_path)

    # open first file
    try:
        hdul1 = fits.open(aia_file_path, mode="update")
        header1 = hdul1[1].header
        data1 = hdul1[1].data
    except Exception as e:
        print(f"FILE CORRUPTED: {aia_file_path}, Error: {e}")
        continue
    
    # Rescale the data to log10 scale AGAIN BECUASE FOR SOME REASON COPYING THE FILE UNDOS IT!!!
    data_log10_1 = np.log10(data1 + 1)  # Adding 1 to avoid log(0)
    # Using pillow or anti-aliasing
    image_pil = Image.fromarray(data_log10_1)
    resized_data_aa = image_pil.resize((224, 224), Image.BILINEAR)
    resized_image_np = np.array(resized_data_aa)
    # Save the downscaled anti-aliased image
    hdul1[1].data = resized_image_np
    
    hdul1.close()
    
    # Open second file
    try:
        hdul2 = fits.open(new_file_path, mode="update")
        header2 = hdul2[1].header
        data2 = hdul2[1].data
    except Exception as e:
        print(f"FILE CORRUPTED: {new_file_path}, Error: {e}")
        continue
    
    # Rescale the data to log10 scale AGAIN BECUASE FOR SOME REASON COPYING THE FILE UNDOS IT!!!
    data_log10_2 = np.log10(data2 + 1)  # Adding 1 to avoid log(0)
    
    # Using OpenCV for non-anti-aliasing
    resized_data = cv2.resize(data_log10_2, (224, 224), interpolation=cv2.INTER_LINEAR)
    # Save the downscaled non-anti-aliased image
    hdul2[1].data = resized_data


    hdul2.close()
    


Downscaling to 224x224:   0%|          | 0/24 [00:00<?, ?it/s]